In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In langrila, the module to chat with LLM and the module to call tools are completely separated. FunctionalChat class is the combination of those two.

- `XXXChatModule`: Only focuses on doing conversation with LLM
- `XXXFunctionCallingModule`: Only focuses on calling tools
- `XXXFunctionalChat`: The combination of the two. FunctionCallingModule works at first and then ChatModule performs. If any tool is not provided, this module behaves as just ChatModule.

In [3]:
from langrila import (
    InMemoryConversationMemory,
    Message,
    PromptTemplate,
    Usage,
)
from langrila.claude import ClaudeFunctionalChat
from langrila.gemini import GeminiFunctionalChat
from langrila.openai import OpenAIFunctionalChat

# Simple text prompt

### For OpenAI Chat Completion

In [4]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="openai",
    # organization_id_env_name="ORGANIZATION_ID",  # as needed
)

In [5]:
prompt = "Hello. How are you today?"

Generate synchronously

In [6]:
response = chat_openai.run(prompt)

response is a pydantic model

In [7]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Hello! I'm just a computer program, so I don't have feelings, but thanks for asking. How can I assist you today?"}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 16,
  'completion_tokens': 26},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Hello. How are you today?'}],
   'name': 'User'}]}

You can pick response text like this

In [8]:
print(response.message.content[0].text)

Hello! I'm just a computer program, so I don't have feelings, but thanks for asking. How can I assist you today?


You can see usage to generate

In [9]:
response.usage

Usage(prompt_tokens=16, completion_tokens=26, total_tokens=42)

You can access prompt

In [10]:
response.prompt

[{'role': 'user',
  'content': [{'type': 'text', 'text': 'Hello. How are you today?'}],
  'name': 'User'}]

Asynchronous call is available.

In [11]:
response = await chat_openai.arun(prompt)
print(response.message.content[0].text)

Hello! I'm here and ready to assist you. How can I help you today?


Streaming completion

In [12]:
stream = chat_openai.stream(prompt)
responses = [r for r in stream]
responses

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello!')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm just")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(

Last response includes usage

In [13]:
responses[-1].usage

Usage(prompt_tokens=16, completion_tokens=34, total_tokens=50)

Asynchronous streaming

In [14]:
stream = chat_openai.astream(prompt)
responses = [r async for r in stream]
responses

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello!')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm just")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(

multiple response generation

In [15]:
response = chat_openai.run(prompt, n_results=2)

In [16]:
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Hello! I don't have feelings, but I'm here and ready to help you. How can I assist you today?"},
  {'text': "Hello! I'm just a machine, so I don't have feelings, but thanks for asking. How can I assist you today?"}],
 'name': None}

In [17]:
# async

response = await chat_openai.arun(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"},
  {'text': "Hello! I'm just a computer program, so I don't have feelings, but thanks for asking. How can I assist you today?"}],
 'name': None}

### For Azure OpenAI

In [18]:
chat_openai_azure = OpenAIFunctionalChat(
    api_key_env_name="AZURE_API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="azure",
    api_version="2024-05-01-preview",
    endpoint_env_name="AZURE_ENDPOINT",
    deployment_id_env_name="AZURE_DEPLOYMENT_ID",
)

In [19]:
response = chat_openai_azure.run(prompt)
print(response.message.content[0].text)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with anything you need. How can I assist you today?


In [20]:
# async
response = await chat_openai_azure.arun(prompt)
print(response.message.content[0].text)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [21]:
# stream
response = chat_openai_azure.stream(prompt)
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello!')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm just")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(

In [22]:
# async stream
response = chat_openai_azure.astream(prompt)
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='Hello!')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="Hello! I'm just")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=[{}]),
 CompletionResults(message=Message(role='assistant', content=[TextContent(

multiple generation

In [23]:
response = chat_openai_azure.run(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Hello! I'm here and ready to assist you. How can I help you today?"},
  {'text': "Hello! I'm just a computer program, so I don't have feelings, but thanks for asking. How can I assist you today?"}],
 'name': None}

In [24]:
# async
response = await chat_openai_azure.arun(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?"},
  {'text': "Hello! I'm here and ready to help you with any questions or tasks you have. How can I assist you today?"}],
 'name': None}

### Gemini on Google Generative AI

In [25]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-flash",
)

In [26]:
response = gemini.run(prompt)
print(response.message.content[0].text)

As an AI, I don't have feelings or experiences like humans do. But I'm here and ready to assist you!  How can I help you today? 😊 



In [27]:
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "As an AI, I don't have feelings or experiences like humans do. But I'm here and ready to assist you!  How can I help you today? 😊 \n"}],
 'name': None}

In [28]:
# async
response = await gemini.arun(prompt)
print(response.message.content[0].text)

I am an AI language model, so I don't have feelings or experiences like humans do. However, I am here to assist you with any questions or tasks you may have. How can I help you today? 😊 



In [29]:
# stream
response = gemini.stream(prompt)
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='As')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="As an AI, I don't have feelings or experiences like humans do. So")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='As an AI, I don\'t have feelings or experiences like humans do. So, I don\'t have a "good" or "bad" day.')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='As an AI, I don\'t have feelings or experiences like humans do. So, I don\'t have a "good" or "bad" day.  However, I\'m here and ready to help you with any questions or tasks you have! 😄 How are you doing today? \n')], na

In [30]:
# async stream
response = gemini.astream(prompt)
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='I')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="I am an AI language model, so I don't have feelings or experiences like")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="I am an AI language model, so I don't have feelings or experiences like humans do. However, I'm here and ready to assist you with any")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="I am an AI language model, so I don't have feelings or experiences like humans do. However, I'm here and ready to assist you with any questions or tasks you may have! 😊 How can I help you

### Gemini on VertexAI

In [31]:
gemini_vertexai = GeminiFunctionalChat(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
)

In [32]:
response = gemini_vertexai.run(prompt)
print(response.message.content[0].text)

As an AI, I don't have feelings or experiences like humans do, so I don't "feel" anything. However, I'm here and ready to assist you with any questions or tasks you may have! 😊 How can I help you today? 



In [33]:
# async
response = await gemini_vertexai.arun(prompt)
print(response.message.content[0].text)

As an AI, I don't have feelings or experiences like humans do. So, I'm not "feeling" anything today. But I'm here and ready to assist you with any questions or tasks you may have! 😊 How can I help you today? 



In [34]:
# stream
response = gemini_vertexai.stream(prompt)
[r for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='As')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="As an AI, I don't have feelings or experiences like humans do, so")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='As an AI, I don\'t have feelings or experiences like humans do, so I don\'t have a "good" or "bad" day. 😊')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='As an AI, I don\'t have feelings or experiences like humans do, so I don\'t have a "good" or "bad" day. 😊  However, I\'m here and ready to assist you with anything you need! How can I help you today? \n')], name=None), u

In [35]:
# async stream
response = gemini_vertexai.astream(prompt)
[r async for r in response]

[CompletionResults(message=Message(role='assistant', content=[TextContent(text='As')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text="As an AI, I don't have feelings or experiences like humans do. So")], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='As an AI, I don\'t have feelings or experiences like humans do. So, I\'m not "feeling" anything in the way you might be asking')], name=None), usage=Usage(prompt_tokens=0, completion_tokens=0, total_tokens=0), prompt=''),
 CompletionResults(message=Message(role='assistant', content=[TextContent(text='As an AI, I don\'t have feelings or experiences like humans do. So, I\'m not "feeling" anything in the way you might be asking. 😊 \n\nHowever, I\'m here and ready to assist you with any questions or 

multiple resposne generation

In [36]:
response = gemini_vertexai.run(prompt, n_results=2)

In [37]:
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "As an AI, I don't have feelings or experiences like humans do. But I'm here and ready to assist you with any questions or tasks you have! 😊 What can I do for you today? \n"},
  {'text': "As an AI, I don't have feelings or experiences like humans do. But I'm here and ready to assist you with any questions or tasks you have! 😊 What can I do for you today? \n"}],
 'name': None}

In [38]:
# async multiple generation
response = await gemini_vertexai.arun(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'As an AI, I don\'t have feelings or experiences like humans do. So, I\'m not "feeling" anything. But I\'m here and ready to assist you with any questions or tasks you may have! 😊 How can I help you today? \n'},
  {'text': 'As an AI, I don\'t have feelings or experiences like humans do. So, I\'m not "feeling" anything in the way you might be. \n\nHowever, I\'m here and ready to assist you with any questions or tasks you have! How can I help you today? 😊 \n'}],
 'name': None}

### For Claude on Anthropic

In [39]:
claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
)

In [40]:
response = claude.run(prompt)
print(response.message.content[0].text)

Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?


In [41]:
# async
response = await claude.arun(prompt)
print(response.message.content[0].text)

Hello! As an AI language model, I don't have feelings, but I'm functioning well and ready to assist you with any questions or tasks you may have. How can I help you today?


Streaming generation have not been supported yet by `ClaudeFunctionalChat`.

### For Claude on Amazon Bedrock

In [42]:
claude_bedrock = ClaudeFunctionalChat(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
)

In [43]:
response = claude_bedrock.run(prompt)
print(response.message.content[0].text)

Hello! As an AI language model, I don't have feelings, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [44]:
response = await claude_bedrock.arun(prompt)
print(response.message.content[0].text)

Hello! As an AI language model, I don't have feelings, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


# Universal message system

Langrila's chat module transforms user prompt and conversation history to universal message object at first, then converts to each client message. Conversation memory has history with universal message format.

In [45]:
prompt = "Please describe this picture."

Converting from prompt to universal message

In [46]:
from langrila.openai import OpenAIMessage

universal_message = OpenAIMessage.to_universal_message(role="user", message=prompt)
universal_message

Message(role='user', content=[TextContent(text='Please describe this picture.')], name=None)

Converting from universal message to client message

In [47]:
OpenAIMessage.to_client_message(universal_message)

{'role': 'user',
 'content': [{'type': 'text', 'text': 'Please describe this picture.'}],
 'name': 'User'}

Gemini module has same interface. Also Claude as well.

In [48]:
from langrila.gemini.genai import GeminiMessage

# same method as that of OpenAIMessage
universal_message = GeminiMessage.to_universal_message(role="user", message=prompt)
universal_message

Message(role='user', content=[TextContent(text='Please describe this picture.')], name=None)

In [49]:
GeminiMessage.to_client_message(universal_message)

parts {
  text: "Please describe this picture."
}
role: "user"

If you want to set a role and name for your prompt, Message class allows you to do it. Raw text prompt is converted to TextContent object in langrila's chat modules, so TextContent object also can be specified.

In [50]:
prompt = Message(role="user", content="Hello.", name="Nsak")
prompt

Message(role='user', content='Hello.', name='Nsak')

Introducing universal message class, we can use OpenAI Chat Completion, Gemini and Claude in a same way and connect them. 

In [51]:
# conversation memory allows you to interact with modules for multi-turn conversation.
shared_memory = InMemoryConversationMemory()

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-flash",
    conversation_memory=shared_memory,
)

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="openai",
    conversation_memory=shared_memory,
)

claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    conversation_memory=shared_memory,
)

Call OpenAI Chat Completion at first.

In [52]:
prompt = "Do you know Rude in Final Fantasy VII?"
response = chat_openai.run(prompt)
print(response.message.content[0].text)

Yes, Rude is a character in Final Fantasy VII, a popular Japanese role-playing game (JRPG) developed by Square Enix. He is a member of the Turks, an elite group within the Shinra Electric Power Company. Rude is known for his stoic demeanor and physical prowess, often serving as the strong, silent type in contrast to his more talkative partner, Reno.

Rude has a distinctive appearance, often wearing sunglasses and a black suit, which is the typical attire of the Turks. Throughout the game, he engages in several battles with the main protagonist group, led by Cloud Strife. Despite his allegiance to Shinra, Rude is depicted as having a sense of honor and often shows a degree of respect towards his adversaries.

Rude's character also appears in other Final Fantasy VII-related media, including the movie "Final Fantasy VII: Advent Children" and the "Final Fantasy VII Remake," where he continues to play his role as a dedicated member of the Turks.


After calling OpenAI Chat Completion, conversation history is inherited by Gemini as it is.

In [53]:
prompt = "What does he think about Tifa?"
response = gemini.run(prompt)
print(response.message.content[0].text)

While there's no explicit statement from Rude about his opinion on Tifa in the original Final Fantasy VII or its spin-offs, we can glean some insights from his interactions with her and his general character traits:

* **Respect for Strength:** Rude is a character who appreciates strength, both physical and mental. He witnesses Tifa's fighting prowess throughout the game and likely respects her abilities. 
* **Professional Detachment:** As a member of the Turks, Rude is trained to be emotionally detached and prioritize his missions. While he might recognize Tifa's strength, it's unlikely he would let personal opinions interfere with his duty. 
* **Potential Antagonism:** Rude, like the other Turks, is loyal to Shinra and sees the protagonists as adversaries. While he may respect Tifa's skills, their positions on opposing sides would likely create a sense of antagonism between them.

**It's important to note that:**

* The original Final Fantasy VII doesn't delve into the Turks' persona

You can see assistant role and message object were automatically transformed for Gemini API in the prompt field.

In [54]:
response.prompt

[parts {
   text: "Do you know Rude in Final Fantasy VII?"
 }
 role: "user",
 parts {
   text: "Yes, Rude is a character in Final Fantasy VII, a popular Japanese role-playing game (JRPG) developed by Square Enix. He is a member of the Turks, an elite group within the Shinra Electric Power Company. Rude is known for his stoic demeanor and physical prowess, often serving as the strong, silent type in contrast to his more talkative partner, Reno.\n\nRude has a distinctive appearance, often wearing sunglasses and a black suit, which is the typical attire of the Turks. Throughout the game, he engages in several battles with the main protagonist group, led by Cloud Strife. Despite his allegiance to Shinra, Rude is depicted as having a sense of honor and often shows a degree of respect towards his adversaries.\n\nRude\'s character also appears in other Final Fantasy VII-related media, including the movie \"Final Fantasy VII: Advent Children\" and the \"Final Fantasy VII Remake,\" where he con

Claude can follow as well.

In [55]:
prompt = "Does Turks members know his feeling?"

response = claude.run(prompt)

print(response.message.content[0].text)

Ah, I see where you're going with this question. You're touching on a subtle but interesting aspect of Rude's character that's hinted at in the game. Let me clarify:

1. In the original Final Fantasy VII, there's a subtle implication that Rude has a crush on Tifa. This isn't explicitly stated, but it's hinted at through some dialogue and interactions.

2. Reno, Rude's partner in the Turks, does seem to be aware of Rude's feelings for Tifa. In one scene, when discussing the AVALANCHE members, Reno teases Rude about liking Tifa.

3. When listing the AVALANCHE members they need to take out, Rude notably omits Tifa's name, which Reno points out teasingly.

4. This crush is never acted upon or explicitly addressed by Rude himself, maintaining his professional demeanor throughout the game.

5. The other Turks' awareness of Rude's feelings isn't clearly established beyond Reno's teasing.

So, to directly answer your question: At least one other Turk (Reno) seems to know about Rude's feelings 

# How to specify system instruction

Langrila's module accepts system instruction as shown below.

### For OpenAI Chat Completion

In [56]:
system_instruction = "You must answer any question only with Yes or No."

In [57]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="openai",
    system_instruction=system_instruction,
    conversation_memory=InMemoryConversationMemory(),
)

In [58]:
prompt = "Please tell me about how to sleep well."
response = chat_openai.run(prompt=prompt)
print(response.message.content[0].text)

No.


System prompt is not stored into conversation memory but it exists in prompt attribute in response object. It's bacause we can make system prompt replaceable flexibly. System prompt is inserted as a first message just before API call.

In [59]:
# You can see system prompt in the prompt attribute in the response
response.model_dump()

{'message': {'role': 'assistant', 'content': [{'text': 'No.'}], 'name': None},
 'usage': {'model_name': 'gpt-4o-2024-05-13',
  'prompt_tokens': 34,
  'completion_tokens': 2},
 'prompt': [{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You must answer any question only with Yes or No.'}],
   'name': 'System'},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': 'Please tell me about how to sleep well.'}],
   'name': 'User'}]}

In [60]:
# System prompt is not shown in the conversation memory
chat_openai.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Please tell me about how to sleep well.'}],
  'name': None},
 {'role': 'assistant', 'content': [{'text': 'No.'}], 'name': None}]

### For Gemini

Gemini and Claude also accept system prompt.

In [61]:
gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-flash",
    system_instruction=system_instruction,
)

In [62]:
prompt = "Please tell me about how to sleep well."
response = gemini.run(prompt=prompt)
print(response.message.content[0].text)

Yes. 



### For Claude

In [63]:
claude = ClaudeFunctionalChat(
    # For Anthropic
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    system_instruction=system_instruction,
)

In [64]:
prompt = "Please tell me about how to sleep well."
response = claude.run(prompt=prompt)
print(response.message.content[0].text)

Yes.


# Token management

For OpenAI Chat Completion, # of tokens is calculated before API call, and module truncates prompt automatically to make sure # of tokens meets token limitation. In this section, we simulate this truncation.

In [65]:
chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="openai",
    context_length=300,  # You can control the number of tokens in the prompt
    conversation_memory=InMemoryConversationMemory(),
)

In [66]:
prompts = [
    "What is the pheasant?",
    "Please tell me about a bird called Kiji in Japan.",
    "What does Kiji do in the tale of the Momotaro",
]

for prompt in prompts:
    response = chat_openai.run(prompt)

Input message is truncated because total length of messages exceeds context length.


Old conversation was truncated as shown below.

In [67]:
# NOTE : Usage after truncation might slightly be different from the specified context_length
response.usage

Usage(prompt_tokens=299, completion_tokens=482, total_tokens=781)

In [68]:
response.prompt

[{'role': 'assistant',
  'content': [{'type': 'text',
    'text': " plumage that helps them blend into their surroundings, particularly when nesting.\n\n### Habitat\n- The Kiji is commonly found in various habitats throughout Japan, including farmlands, grasslands, forests, and urban areas. They are well-adapted to the Japanese environment, benefiting from agricultural lands which provide ample food sources.\n\n### Behavior\n- **Diet:** Kiji primarily forage on the ground, eating seeds, grains, insects, and small invertebrates.\n- **Flight:** They are ground birds but can make short, strong flights if startled.\n- **Breeding:** During the breeding season, males become quite territorial and display their colorful plumage to attract females. They are polygynous, meaning one male may mate with several females.\n\n### Cultural Significance\n- The Japanese Pheasant holds cultural significance in Japan and appears in various forms of art and folklore. It is often featured in traditional Japa

But truncation does not affect conversation memory itself. You can see full conversation history in the memory.

In [69]:
chat_openai.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'What is the pheasant?'}],
  'name': None},
 {'role': 'assistant',
  'content': [{'text': 'The pheasant is a type of bird belonging to the family Phasianidae, which also includes partridges, chickens, quails, and peafowl. Pheasants are known for their bright plumage and elongated tails. The most well-known species is the Common Pheasant (Phasianus colchicus), native to Asia but widely introduced around the world for hunting and ornamental purposes.\n\nMale pheasants, known as roosters or cocks, are typically more colorful than females, featuring iridescent feathers in shades of gold, green, purple, and red. They also often have distinctive markings and long, elegant tails. Female pheasants, or hens, have more subdued, camouflaged plumage, which helps them blend into their environment, especially when nesting.\n\nPheasants are typically ground-dwelling birds that prefer a mix of open fields and dense cover, such as hedgerows or woodlands. They fo

If prompts contain some images and # of tokens exceeds limitation, image might be truncated entirely.

# Usage gathering across multiple models

Usage object allows you to sum multiple usages.

In [70]:
usage1 = Usage(model_name="gpt-4o-2024-05-13", prompt_tokens=100, completion_tokens=50)

usage2 = Usage(model_name="gpt-4o-2024-05-13", prompt_tokens=200, completion_tokens=100)

usage1 + usage2

Usage(prompt_tokens=300, completion_tokens=150, total_tokens=450)

Usage don't have capability to sum usages from different models.

In [71]:
usage3 = Usage(model_name="gpt-4o-mini-2024-07-18", prompt_tokens=300, completion_tokens=150)

try:
    usage1 + usage3  # Error
except:
    print("Error!")

Error!


But usage with empty model_name is added to any other usage.

In [72]:
usage4 = Usage(prompt_tokens=100, completion_tokens=50)

usage1 + usage4

Usage(prompt_tokens=200, completion_tokens=100, total_tokens=300)

On the other hand, it's not possible to add usage to usage of empty model_name.

In [73]:
try:
    usage4 + usage1
except:
    print("Error!")

Error!


Using this function, we sometimes want to sum up total #of tokens over each models. In langrila, shared token counter is available to integrate total usage from multiple models.

In [74]:
from langrila import TokenCounter

shared_memory = InMemoryConversationMemory()

shared_token_counter = TokenCounter()

gemini = GeminiFunctionalChat(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-flash",
    conversation_memory=shared_memory,
    token_counter=shared_token_counter,
)

chat_openai = OpenAIFunctionalChat(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-2024-05-13",
    api_type="openai",
    conversation_memory=shared_memory,
    token_counter=shared_token_counter,
)

claude = ClaudeFunctionalChat(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    conversation_memory=shared_memory,
    token_counter=shared_token_counter,
)

In [75]:
prompts = [
    "What is the pheasant?",
    "Please tell me about a bird called Kiji in Japan.",
    "What does Kiji do in the tale of the Momotaro",
]

clients = [gemini, chat_openai, claude]

for i, prompt in enumerate(prompts):
    client = clients[i]
    response = client.run(prompt)

In [76]:
shared_token_counter

{'gemini-1.5-flash': Usage(prompt_tokens=6, completion_tokens=457, total_tokens=463), 'gpt-4o-2024-05-13': Usage(prompt_tokens=506, completion_tokens=477, total_tokens=983), 'claude-3-5-sonnet-20240620': Usage(prompt_tokens=1104, completion_tokens=416, total_tokens=1520)}

# Prompt template

We can manage a typical prompt as a template.

In [77]:
template = PromptTemplate.from_text_file("../data/sample_prompt_template.txt")
print(template.template)

# INSTRUCTION
{instruction}

# TEXT
{text}


In [78]:
template.set_args(instruction="Hello", text="world!")

PromptTemplate(args={'instruction': 'Hello', 'text': 'world!'}, template='# INSTRUCTION\n{instruction}\n\n# TEXT\n{text}')

In [79]:
print(template.format())

# INSTRUCTION
Hello

# TEXT
world!


We can pass template string to PromptTemplate directly.

In [80]:
PromptTemplate(template="# INSTRUCTION\n{instruction}\n\n# TEXT\n{text}")

PromptTemplate(args={}, template='# INSTRUCTION\n{instruction}\n\n# TEXT\n{text}')